In this notebook we build the MLGraph for the Amsterdam pilot from the data. 

## Imports

In [ ]:
### Imports 
import os
import math
import sys
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib.patches as patches

import pandas as pd
import re
import xml.etree.ElementTree as ET

sys.path.append('../../')
from script.conversion.bison.coordinates import rd_to_utm
from mnms.graph.layers import PublicTransportLayer, MultiLayerGraph
from mnms.generation.roads import generate_pt_line_road, generate_one_zone
from mnms.generation.layers import generate_bbox_origin_destination_layer
from mnms.vehicles.veh_type import Tram, Metro, Bus
from mnms.generation.zones import generate_one_zone
from mnms.mobility_service.public_transport import PublicTransportMobilityService
from mnms.time import TimeTable, Dt, Time
from mnms.io.graph import load_graph, save_graph
from mnms.tools.render import draw_roads, draw_line, draw_odlayer
from mnms.tools.geometry import points_in_polygon, get_bounding_box

## Parameters

In [ ]:
### Parameters

# Files and directories
current_dir = os.getcwd()
indir = current_dir + '/inputs/'
outdir = current_dir + '/outputs/'

coord_csv_filepath = indir + 'KV1_GVB_2609_2/Csv/POINT.csv' # file with coordinates of the network
amsterdam_json_filepath = indir + 'new_network.json' # mlgraph with the road network only

metro_xml_directory = indir + 'KV1_GVB_2609_2/Xml/METRO' # Definition of operation patterns for METRO lines
tram_xml_directory = indir + 'KV1_GVB_2609_2/Xml/TRAM' # Definition of operation patterns for TRAM lines
bus_xml_directory = indir + 'KV1_GVB_2609_2/Xml/BUS' # Definition of operation patterns for BUS lines
ns = 'http://bison.connekt.nl/tmi8/kv1/msg' # something related to xml domain

# Points coordinates
df_points = pd.read_csv(coord_csv_filepath, sep='|', converters={'DATAOWNERCODE': str})
df_points = df_points[['DATAOWNERCODE', 'LOCATIONXEW', 'LOCATIONYNS']]

# Default speeds
traditional_vehs_default_speed = 13.8 # m/s
metro_default_speed = 15 # m/s
tram_default_speed = 18 # m/s

# PT operation parameters
bus_freq = Dt(minutes=8)
metro_freq = Dt(minutes=4)
tram_freq = Dt(minutes=10)
operation_start_time = '05:00:00'
operation_end_time = '12:00:00'

# Roads 
roads_polygon = [[616900, 5792000],
                 [641900, 5792000],
                 [641900, 5813000],
                 [616900, 5813000]]

# Origin-destination layer 
od_mesh_size = 250 # m
od_layer_polygon = [[620000,5798000],[630000,5798000], [632500, 5794000], [637000, 5797600], 
                    [637900, 5802000], [635000, 5804500], [633000, 5809000], [628000, 5811300], 
                    [624000, 5810000], [620000, 5810000], [617500, 5802500]]

# Layers connection parameters 
max_access_egress_dist = 500 # m

## Functions

In [ ]:
### Functions

_norm = np.linalg.norm

def extract_amsterdam_stops(xml_dir):
    """Function that create the list of lines for one public transportation type. 
    WARNING: this function is made simple, it cannot deal with different operation patterns for one line, and could 
    not deal with BUS 232 and BUS 233 which have specific operation patterns. We removed these two lines from the data 
    for now. 
    
    Args: 
    - xml_dir: directory where the xml files for each line of a public transportation types are located 
    
    Returns: 
    - list_lines: list of lines, a line is represented by a dataframe with LINE_ID, STOP_NAME, and STOP_CODE as columns
    """
    # Get all files in xml_dir, there should be one file per public transportation line
    files = os.listdir(xml_dir)
    files_xml = list(filter(lambda f: f.endswith('.xml'), files))
    
    # Init the list of lines
    list_lines = []

    # Build each line 
    for file in files_xml:
        xml_tree = ET.parse(xml_dir + "/" + file)

        stop_list = xml_tree.findall(".//{*}USRSTOPbegin")
        last_stop = xml_tree.findall(".//{*}USRSTOPend")[-1]
        stop_list.append(last_stop)
        
        line_number = file.replace(".xml", '') # the line number is the file name 

        line_stops = []
        
        # Get the stop list for this line 
        # NB: this part of the code supposes that we have only one operation pattern defined in the file
        for stop in stop_list:
            userstopcode = ""
            name = ""

            for child in stop:
                if child.tag == "{" + ns + "}userstopcode":
                    userstopcode = child.text
                if child.tag == "{" + ns + "}name":
                    name = re.sub(r'\W+', '', child.text)
            line_stops.append({'LINE_ID': line_number, 'STOP_NAME': name, 'STOP_CODE': userstopcode})
        
        df_line = pd.DataFrame(line_stops, columns=["LINE_ID", "STOP_NAME", "STOP_CODE"])
        
        # Seperate the two directions of this line 
        # NB: this part of the code supposes that the second stop in one direction and the penultimate stop in the other direction are the same !
        for i in range(1,len(df_line)-1):
            if df_line.iloc[i-1]['STOP_NAME'] == df_line.iloc[i+1]['STOP_NAME']:
                df_line_dir1 = df_line.iloc[:i+1]
                df_line_dir1 = df_line_dir1.assign(LINE_ID=df_line_dir1.iloc[0]['LINE_ID'] + '_DIR1')
                df_line_dir1.reset_index(drop=True, inplace=True)
                df_line_dir2 = df_line.iloc[i:]
                df_line_dir2 = df_line_dir2.assign(LINE_ID=df_line_dir2.iloc[0]['LINE_ID'] + '_DIR2')
                df_line_dir2.reset_index(drop=True, inplace=True)
    
        # Append two lines, one per direction 
        list_lines.append(df_line_dir1)
        list_lines.append(df_line_dir2)
        
    return list_lines

def generate_public_transportation_lines(layer, list_lines, freq, operation_start_time, operation_end_time, prefix_line_name):
    """Function that generates public transportation lines on a layer with a certain frequency. 
    
    Args: 
    - layer: the layer on which the lines should be created
    - list_lines: list the lines to create, one line is represented by a dataframe with LINE_ID, STOP_NAME, and STOP_CODE as columns
    - freq: frequency to apply on the lines created, shoud be a Dt type object
    - operation_start_time: time at which the timetables should start, str
    - operation_end_time: time at which the timetables should end, str
    - prefix_line_name: str corresponding to the prefix to add to the line id to name the line
    
    Returns: 
    None
    """
    for line in list_lines:
        line_id = line.iloc[0]['LINE_ID']
        line_name = prefix_line_name + line_id
        stops = [line_name + f'_{ind}' for ind in line.index]
        sections = [[line_name + f'_{ind}_{ind+1}', line_name + f'_{ind+1}_{ind+2}'] for ind in list(line.index)[:-2]] + [[line_name + f'_{line.index[-2]}_{line.index[-1]}']]
        layer.create_line(line_name, stops, sections, TimeTable.create_table_freq(operation_start_time, operation_end_time, freq))
        
        
def keep_roads_in_polygon(roads, polygon):
    """Function that clean all sections and nodes that are outside of a certain polygon. 
    
    Args:
    - roads: a RoadDescriptor object 
    - polygon: a PointList object that defines a polygon 
    
    Return: 
    None
    """
    nodes_pos = [np.array(n.position) for n in roads.nodes.values()]
    nodes_ids = np.array([n for n in roads.nodes])
    polygon_array = np.array(polygon)
    mask = points_in_polygon(polygon_array, nodes_pos)
    nodes_kept = nodes_ids[mask].tolist()
    nodes_to_remove = []
    for nid in roads.nodes.keys():
        if nid not in nodes_kept:
            nodes_to_remove.append(nid)
    roads.delete_nodes(nodes_to_remove)

## Roads

In [ ]:
### Get the lines definition from the data
lines = []    
list_bus_lines = extract_amsterdam_stops(bus_xml_directory)
list_tram_lines = extract_amsterdam_stops(tram_xml_directory)
list_metro_lines = extract_amsterdam_stops(metro_xml_directory) # /!\ keep only one JOPA for each direction in the data file
                                                                # JOPA correspond to different operation patterns, eg express train that do not stop at certain stations

In [ ]:
### Get the MLGraph without TCs 
amsterdam_graph = load_graph(amsterdam_json_filepath)
roads = amsterdam_graph.roads

### Clean roads by removing far isolated sections 
keep_roads_in_polygon(roads, roads_polygon)

### Add the nodes, sections, and stops related to each PT line to the roadDescriptor
pt_lines = list_tram_lines + list_metro_lines + list_bus_lines
pt_lines_types = ['TRAM'] * len(list_tram_lines) + ['METRO'] * len(list_metro_lines) + ['BUS'] * len(list_bus_lines)
pt_nodes = {}
for line, line_type in zip(pt_lines, pt_lines_types):
    for ind, stop in line.iterrows():
        x = float(df_points["LOCATIONXEW"].loc[df_points["DATAOWNERCODE"] == stop['STOP_CODE']])
        y = float(df_points["LOCATIONYNS"].loc[df_points["DATAOWNERCODE"] == stop['STOP_CODE']])
        x_utm, y_utm = rd_to_utm(x, y)
        node_id = line_type + '_' + stop['LINE_ID'] + '_' + str(ind)
        pt_nodes[node_id] = [x_utm,y_utm]
        roads.register_node(node_id, [x_utm, y_utm])
        if ind > 0:
            onode_id = line_type + '_' + stop['LINE_ID'] + '_' + str(ind-1)
            dnode_id = node_id
            section_id = line_type + '_' + stop['LINE_ID'] + '_' + str(ind-1) + '_' + str(ind)
            section_length = _norm(np.array(pt_nodes[onode_id]) - np.array(pt_nodes[dnode_id]))
            roads.register_section(section_id, onode_id, dnode_id, section_length)
            roads.register_stop(onode_id, section_id, 0.)
        if ind == max(line.index):
            roads.register_stop(dnode_id, section_id, 1.)

In [ ]:
### Overwrite the roads zoning with a new zoning including all sections
roads.add_zone(generate_one_zone("RES", roads))

## MLGraph

In [ ]:
### Create the PT layers, mob services and lines

# Bus
bus_service = PublicTransportMobilityService('BUS')
bus_layer = PublicTransportLayer(roads, 'BUSLayer', Bus, traditional_vehs_default_speed,
        services=[bus_service])
generate_public_transportation_lines(bus_layer, list_bus_lines, bus_freq, operation_start_time, operation_end_time, 'BUS_')
# Metro
metro_service = PublicTransportMobilityService('METRO')
metro_layer = PublicTransportLayer(roads, 'METROLayer', Metro, metro_default_speed,
        services=[metro_service])
generate_public_transportation_lines(metro_layer, list_metro_lines, metro_freq,operation_start_time, operation_end_time, 'METRO_')
# Tram
tram_service = PublicTransportMobilityService('TRAM')
tram_layer = PublicTransportLayer(roads, 'TRAMLayer', Tram, tram_default_speed,
        services=[tram_service])
generate_public_transportation_lines(tram_layer, list_tram_lines, tram_freq, operation_start_time, operation_end_time, 'TRAM_')

In [ ]:
### Create the OD layer
bbox = get_bounding_box(roads)
nx = int((bbox.xmax - bbox.xmin ) / od_mesh_size)
ny = int((bbox.ymax - bbox.ymin ) / od_mesh_size)
od_layer = generate_bbox_origin_destination_layer(roads, nx, ny, od_layer_polygon)

In [ ]:
### Create the MLGraph with PT
mlgraph = MultiLayerGraph([bus_layer, tram_layer, metro_layer], od_layer, max_access_egress_dist)

## Visualization

In [ ]:
### Visualize the roads and ODs 
fig, ax = plt.subplots(figsize=(15, 15))
draw_roads(ax, roads, color='grey', linkwidth=0.1, nodesize=0, draw_stops=False, node_label=False)
draw_odlayer(ax, mlgraph, color='black', nodesize=0.5, node_label=False)

# Draw rectangles to help clean the roads 
#rectangles = {r'rec': patches.Rectangle((616900, 5792000), 25000, 21000, linewidth=8, edgecolor='gold', facecolor='none')}
#for recname, rec in rectangles.items():
#    ax.add_patch(rec)
#plt.show()

# Draw polygon to help clean ODLayer
#polygons = {r'pol': patches.Polygon([[620000,5798000],[630000,5798000], [632500, 5794000], [637000, 5797600], 
#                                     [637900, 5802000], [635000, 5804500], [633000, 5809000], [628000, 5811300], 
#                                     [624000, 5810000], [620000, 5810000], [617500, 5802500]], linewidth=8, edgecolor='gold', facecolor='none')}
#for polname, pol in polygons.items():
#    ax.add_patch(pol)

#plt.show()
plt.savefig(outdir + 'amsterdam_roads_ods.pdf', bbox_inches='tight')

In [ ]:
### Visuaize the whole network 

# Params of the visualization
colors = {'BUS': 'green', 'METRO': 'red', 'TRAM': 'skyblue'}

fig, ax = plt.subplots(figsize=(15, 15))
draw_roads(ax, mlgraph.roads, color='grey', linkwidth=0.1, nodesize=0, draw_stops=False, node_label=False)

for layer in mlgraph.layers.values():
    if type(layer) == PublicTransportLayer:
        for name, line in layer.lines.items():
                draw_line(ax, mlgraph, line, color=colors[name[:name.find('_')]], 
                          linkwidth=0.4, nodesize=1, line_label=None, label_size=1, alpha=1., stopmarkeredgewidth=0.1)

legend = [Line2D([0, 1], [0, 1], marker='.', markersize=12, markeredgecolor='black', color='skyblue', linewidth=5,
            label='Tram line'),
          Line2D([0, 1], [0, 1], marker='.', markersize=12, markeredgecolor='black', color='red', linewidth=5,
            label='Metro line'),
          Line2D([0, 1], [0, 1], marker='.', markersize=12, markeredgecolor='black', color='green', linewidth=5,
            label='Bus line'),
          Line2D([0, 1], [0, 1], marker='.', markersize=0, markeredgecolor='grey', markerfacecolor='grey', color='black', linewidth=1,
            label='Roads')]

legend = plt.legend(handles=legend)
legend.get_frame().set_alpha(0.92)

#plt.show()
plt.savefig(outdir + 'amsterdam_network.pdf', bbox_inches='tight')